In [1]:
import pandas as pd
train_df = pd.read_csv("../sales_train.csv.gz")
test_df  = pd.read_csv("../test.csv.gz")

In [2]:
%store -r __prepare_data

In [3]:
__prepare_data

In [4]:
all_data = all_data[all_data.date_block_num > 0] # no lags before time 0
all_data.fillna(0, inplace=True)
print(all_data.isnull().values.any())

False


In [5]:
from sklearn.model_selection import train_test_split

target = all_data.target.values
target[target>20] = 20

predictors = ["target_lag_1","item_target_enc_lag_1","shop_target_enc_lag_1",
                             "item_target_enc_ave",  "shop_target_enc_ave"]

X_train, X_test, y_train, y_test = \
    train_test_split(all_data[predictors], target, test_size=0.2, random_state=321)

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate

model  = RandomForestRegressor(n_estimators=32, max_features=2, n_jobs=2) #len(predictors)/2
cv = cross_validate(model, X_train, y_train, cv=5, scoring="neg_mean_squared_error", n_jobs=2, verbose=10)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed: 31.4min
[Parallel(n_jobs=2)]: Done   3 out of   5 | elapsed: 62.3min remaining: 41.5min
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed: 79.0min finished
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed: 79.0min remaining:    0.0s


In [8]:
-cv['test_score']

array([0.6812877 , 0.6797963 , 0.68570715, 0.68499971, 0.6827028 ])

In [9]:
from sklearn.metrics import mean_squared_error

model2 = RandomForestRegressor(n_estimators=32, max_features=2, n_jobs=2) #len(predictors)/2
model2.fit(X_train, y_train)
ypred = model2.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, ypred))
print("RMSE: %f" % (rmse))

RMSE: 0.825290


In [10]:
full_model = RandomForestRegressor(n_estimators=32, max_features=2, n_jobs=2) #len(predictors)/2
full_model.fit(all_data[predictors], target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=2, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=32, n_jobs=2, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [17]:
%store -r __prepare_submission

In [35]:
__prepare_submission

In [38]:
aliases = \
{"target_lag_1":          "target",
 "item_target_enc_lag_1": "item_target_enc",
 "shop_target_enc_lag_1": "shop_target_enc",
 "item_target_enc_ave":   "item_target_enc_ave",
 "shop_target_enc_ave":   "shop_target_enc_ave"
 }

prepare_submission(full_model, predictors, aliases, "skl_rf_test.csv")